v4: Only 2 variables

v5: with linear regression on RL

In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)


import warnings
warnings.filterwarnings("ignore")

In [26]:
data = pd.read_csv('edited_train.csv')
save_for_later = pd.read_csv('save_for_later.csv')
song_labels = pd.read_csv('song_labels.csv')
save_for_later['count'] = 1


song_labels = song_labels.drop_duplicates(subset=['label_id', 'platform_id'], keep='last')
song_labels = song_labels.pivot(index='platform_id', columns='label_id', values='count')
save_for_later = save_for_later.pivot(index='customer_id', columns='song_id', values='count')
train_pivot = train.pivot(index='customer_id', columns='song_id', values='score')


X  = np.nan_to_num(song_labels, nan = 0)
pca = PCA(n_components=100)
X = pca.fit_transform(X)
song_labels = pd.DataFrame(X, index = song_labels.index)

X  = np.nan_to_num(save_for_later, nan = 0)
pca = PCA(n_components=19)
X = pca.fit_transform(X)
save_for_later = pd.DataFrame(X, index = save_for_later.index)

X  = np.nan_to_num(train_pivot, nan = 0)
pca = PCA(n_components=67)
X = pca.fit_transform(X)
train_pivot = pd.DataFrame(X, index = train_pivot.index)


train, val = train_test_split(data, test_size=0.05, random_state=42, stratify = data['score'])

def merge_feature(train, df, feat_m, feat_trans, name, func, test = False):
    if func == 1:
        edited = train.groupby([feat_m]).count()
    if func == 2:
        edited = train.groupby([feat_m]).var()
    if func == 3:
        edited = train.groupby([feat_m]).std()
    if func == 4:
        edited = train.groupby([feat_m]).mean()
        
    edited = edited.reset_index()
    df = df.merge(edited[[feat_m, feat_trans]], how = 'left' , on = feat_m)
    if test == False:
        df = df.rename(columns ={feat_trans + '_x': feat_trans, feat_trans+'_y': name})
    else:
        df = df.rename(columns ={feat_trans : name})
    

    return df

train = merge_feature(train, train, 'customer_id', 'song_id','cust_count', 1)
val = merge_feature(train, val, 'customer_id', 'song_id','cust_count', 1)
train = merge_feature(train, train, 'customer_id', 'score','cust_var', 2)
val = merge_feature(train, val, 'customer_id', 'score','cust_var', 2)
train = merge_feature(train, train, 'customer_id', 'score','cust_std', 3)
val = merge_feature(train, val, 'customer_id', 'score','cust_std', 3)
# train = merge_feature(save_for_later, train, 'customer_id', 'song_id','later_usercount', 1)
# val = merge_feature(save_for_later, val, 'customer_id', 'song_id','later_usercount', 1)


train = merge_feature(train, train, 'song_id', 'customer_id','song_count', 1)
val = merge_feature(train, val, 'song_id', 'customer_id','song_count', 1)
train = merge_feature(train, train,'song_id', 'score','song_var', 2)
val = merge_feature(train, val, 'song_id', 'score','song_var', 2)
train = merge_feature(train, train,'song_id', 'score','song_std', 3)
val = merge_feature(train, val, 'song_id', 'score','song_std', 3)
# train = merge_feature(save_for_later, train, 'song_id', 'customer_id','later_songcount', 1)
# val = merge_feature(save_for_later, val, 'song_id', 'customer_id','later_songcount', 1)
# train = merge_feature(song_labels, train, 'platform_id', 'label_id','label_count', 1, True)
# val = merge_feature(song_labels, val, 'platform_id', 'label_id','label_count', 1, True)
# train = merge_feature(song_labels, train, 'platform_id', 'count','label_avg', 4, True)
# val = merge_feature(song_labels, val, 'platform_id', 'count','label_avg', 4, True)

# train_ratings = train.groupby(['customer_id', 'score']).agg({'score': ['count']})
# train_ratings.columns=['num_rate']
# train_ratings=train_ratings.reset_index()
# train_ratings["user_rate_1"] = train_ratings["num_rate"]
# train_ratings.loc[train_ratings['score'] != 1, 'user_rate_1'] = 0
# train_ratings=train_ratings.drop(['score'],axis=1)
# train_ratings=train_ratings.drop(['num_rate'],axis=1)
# edited = train_ratings.groupby(['customer_id']).sum()
# train = pd.merge(train, edited, on='customer_id', how='left')
# val = pd.merge(val, edited, on='customer_id', how='left')

# train_ratings = train.groupby(['song_id', 'score']).agg({'score': ['count']})
# train_ratings.columns=['num_rate']
# train_ratings=train_ratings.reset_index()
# train_ratings["song_rate_1"] = train_ratings["num_rate"]
# train_ratings.loc[train_ratings['score'] != 1, 'song_rate_1'] = 0
# train_ratings=train_ratings.drop(['score'],axis=1)
# train_ratings=train_ratings.drop(['num_rate'],axis=1)
# edited = train_ratings.groupby(['song_id']).sum()
# train = pd.merge(train, edited, on='song_id', how='left')
# val = pd.merge(val, edited, on='song_id', how='left')

train=train.fillna(0)
val=val.fillna(0)

Y_train = train.score
# X_train = train.drop(['score','customer_id', 'song_id', 'platform_id' ,'language'], axis =1)
# X_test = val.drop(['score','customer_id', 'song_id', 'platform_id','language'], axis =1)
X_train = train[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]
X_test = val[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]

song_labels = song_labels.reset_index()
X_train = X_train.merge(song_labels, how = 'left' , on = 'platform_id')
X_train = X_train.drop(['platform_id'],axis=1)

X_train = X_train.merge(save_for_later, how = 'left' , on = 'customer_id')
X_train = X_train.merge(train_pivot, how = 'left' , on = 'customer_id')
X_train = X_train.drop(['customer_id'],axis=1)


X_test = X_test.merge(song_labels, how = 'left' , on = 'platform_id')
X_test = X_test.drop(['platform_id'],axis=1)

X_test = X_test.merge(save_for_later, how = 'left' , on = 'customer_id')
X_test = X_test.merge(train_pivot, how = 'left' , on = 'customer_id')
X_test = X_test.drop(['customer_id'],axis=1)

model =  xgb.XGBRegressor(learning_rate = 0.1, verbosity = 1)
model.fit(X_train, Y_train)

val['RF_preds'] = model.predict(X_test)

In [27]:
songs_df=pd.read_csv('songs.csv')
test = val
df_submission = pd.read_csv('dummy_submission.csv')
Y_test = test.score
test = test.drop(['score'], axis=1)

In [28]:
test.head()

,customer_id,song_id,First_Customer_id,platform_id,released_year,language,number_of_comments,First_song_id,song_avg,cust_avg,cust_count,cust_var,cust_std,song_count,song_var,song_std,RF_preds
0,J49544,49,J,Q49041,2006.0,eng,44020.0,Q,3.202439,3.588235,64,1.299355,1.139893,1167,1.591164,1.261414,2.355881
1,L41166,859,L,S3227063,2008.0,en-US,4025.0,S,4.150794,5.000000,18,0.000000,0.000000,117,0.700265,0.836819,4.986087
2,J21804,1950,J,X102458,1994.0,eng,618.0,X,4.192308,4.643836,67,0.298055,0.545944,51,0.628235,0.792613,4.733507
3,N3398,6808,N,W15047,1987.0,eng,1035.0,W,4.240000,3.842105,53,0.771408,0.878298,22,0.885281,0.940894,4.156301
4,M18747,2170,M,P16790,1998.0,eng,2160.0,P,4.175676,3.181818,50,0.762857,0.873417,67,0.613297,0.783133,3.444031


In [29]:
user_ids = train.customer_id.unique()
total_users = len(user_ids)

In [30]:
### TRUNCSVD ###

#Removing duplicates
song_matrix = pd.concat([train,test]).drop_duplicates(subset = ['customer_id','song_id'],keep = 'first')
#Creates a song matrix of #numofusers vs #noofsongs
song_matrix = song_matrix.pivot('customer_id','song_id','score')

song_means = song_matrix.mean()
user_means = song_matrix.mean(axis=1)
#Mean shifting
song_shifted_temp = song_matrix-song_means
song_shifted = song_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -song_shifted_temp.isnull()

def repeated_matrix_reconstruction(num_pcs,num_iterations):
    global song_shifted
    for i in range(num_iterations):
        SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
        SVD.fit(song_shifted)
        #For the ease of applying masks we work with pandas
        song_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(song_shifted)),columns=song_shifted.columns,index=song_shifted.index)
        loss = mean_squared_error(song_represented[mask].fillna(0),song_shifted_temp[mask].fillna(0))
        print('Iteration: {} , Loss: {} '.format(i,loss))
        #To just update the non-zero values of song_reprented values to the true ratings
        
        if i < (num_iterations - 1):
            song_represented[mask] = song_shifted_temp[mask]
        
        song_shifted = song_represented
            
    #Mean shifting it back
    song_mat = song_shifted + song_means
    song_mat = song_mat.clip(lower=1,upper=5)
    return song_mat
print("Starting truncated svd with number of components as 20")
representative_matrix_20 = repeated_matrix_reconstruction(50,30)
print("Done")
print("Starting truncated svd with number of components as 15")
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
print("Done")
#bagging
rating_matrix = (representative_matrix_15+representative_matrix_20)/2

Starting truncated svd with number of components as 20
Iteration: 0 , Loss: 0.0035972677867262665 
Iteration: 1 , Loss: 0.0033787619016398584 
Iteration: 2 , Loss: 0.0031921679992499992 
Iteration: 3 , Loss: 0.0030325716114350627 
Iteration: 4 , Loss: 0.0028945850745326464 
Iteration: 5 , Loss: 0.002773645770605444 
Iteration: 6 , Loss: 0.0026663030785131566 
Iteration: 7 , Loss: 0.002569965369017499 
Iteration: 8 , Loss: 0.0024826667939619683 
Iteration: 9 , Loss: 0.0024028989439441083 
Iteration: 10 , Loss: 0.0023294903653921727 
Iteration: 11 , Loss: 0.0022615196091212153 
Iteration: 12 , Loss: 0.002198252208542089 
Iteration: 13 , Loss: 0.0021390948824412193 
Iteration: 14 , Loss: 0.0020835621356715705 
Iteration: 15 , Loss: 0.0020312517644129106 
Iteration: 16 , Loss: 0.001981826751512683 
Iteration: 17 , Loss: 0.0019350017549771576 
Iteration: 18 , Loss: 0.001890532911965898 
Iteration: 19 , Loss: 0.0018482100516168457 
Iteration: 20 , Loss: 0.0018078506726484766 
Iteration: 21 ,

In [31]:
trunc_prediction = np.zeros(len(test))
for i in range(len(test)):
    userid =  test.iloc[i,0]
    songid = test.iloc[i,1]
    trunc_prediction[i] = rating_matrix[rating_matrix.index==userid][songid].values[0]
    
val['RM_preds'] = trunc_prediction

In [32]:
X_train_final = val[['RM_preds','RF_preds']]
Y_train_final = Y_test

final_model =  LinearRegression()
final_model.fit(X_train_final, Y_train_final)

LinearRegression()

In [33]:
save_val = val

train = pd.read_csv('edited_train.csv')
val = pd.read_csv('edited_test.csv') 
test = pd.read_csv('test.csv')


train = merge_feature(train, train, 'customer_id', 'song_id','cust_count', 1)
val = merge_feature(train, val, 'customer_id', 'song_id','cust_count', 1)
train = merge_feature(train, train, 'customer_id', 'score','cust_var', 2)
val = merge_feature(train, val, 'customer_id', 'score','cust_var', 2, True)
train = merge_feature(train, train, 'customer_id', 'score','cust_std', 3)
val = merge_feature(train, val, 'customer_id', 'score','cust_std', 3, True)
# train = merge_feature(save_for_later, train, 'customer_id', 'song_id','later_usercount', 1)
# val = merge_feature(save_for_later, val, 'customer_id', 'song_id','later_usercount', 1)



train = merge_feature(train, train, 'song_id', 'customer_id','song_count', 1)
val = merge_feature(train, val, 'song_id', 'customer_id','song_count', 1)
train = merge_feature(train, train,'song_id', 'score','song_var', 2)
val = merge_feature(train, val, 'song_id', 'score','song_var', 2, True)
train = merge_feature(train, train,'song_id', 'score','song_std', 3)
val = merge_feature(train, val, 'song_id', 'score','song_std', 3, True)
# train = merge_feature(save_for_later, train, 'song_id', 'customer_id','later_songcount', 1)
# val = merge_feature(save_for_later, val, 'song_id', 'customer_id','later_songcount', 1)
# train = merge_feature(song_labels, train, 'platform_id', 'label_id','label_count', 1, True)
# val = merge_feature(song_labels, val, 'platform_id', 'label_id','label_count', 1, True)
# train = merge_feature(song_labels, train, 'platform_id', 'count','label_avg', 4, True)
# val = merge_feature(song_labels, val, 'platform_id', 'count','label_avg', 4, True)


# train_ratings = train.groupby(['customer_id', 'score']).agg({'score': ['count']})
# train_ratings.columns=['num_rate']
# train_ratings=train_ratings.reset_index()
# train_ratings["user_rate_1"] = train_ratings["num_rate"]
# train_ratings.loc[train_ratings['score'] != 1, 'user_rate_1'] = 0
# train_ratings=train_ratings.drop(['score'],axis=1)
# train_ratings=train_ratings.drop(['num_rate'],axis=1)
# edited = train_ratings.groupby(['customer_id']).sum()
# train = pd.merge(train, edited, on='customer_id', how='left')
# val = pd.merge(val, edited, on='customer_id', how='left')

# train_ratings = train.groupby(['song_id', 'score']).agg({'score': ['count']})
# train_ratings.columns=['num_rate']
# train_ratings=train_ratings.reset_index()
# train_ratings["song_rate_1"] = train_ratings["num_rate"]
# train_ratings.loc[train_ratings['score'] != 1, 'song_rate_1'] = 0
# train_ratings=train_ratings.drop(['score'],axis=1)
# train_ratings=train_ratings.drop(['num_rate'],axis=1)
# edited = train_ratings.groupby(['song_id']).sum()
# train = pd.merge(train, edited, on='song_id', how='left')
# val = pd.merge(val, edited, on='song_id', how='left')

train=train.fillna(0)
val=val.fillna(0)

Y_train = train.score
# X_train = train.drop(['score','customer_id', 'song_id', 'platform_id' ,'language'], axis =1)
# X_test = val.drop(['score','customer_id', 'song_id', 'platform_id','language'], axis =1)
X_train = train[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]
X_test = val[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]

X_train = X_train.merge(song_labels, how = 'left' , on = 'platform_id')
X_train = X_train.drop(['platform_id'],axis=1)

X_train = X_train.merge(save_for_later, how = 'left' , on = 'customer_id')
X_train = X_train.merge(train_pivot, how = 'left' , on = 'customer_id')
X_train = X_train.drop(['customer_id'],axis=1)


X_test = X_test.merge(song_labels, how = 'left' , on = 'platform_id')
X_test = X_test.drop(['platform_id'],axis=1)

X_test = X_test.merge(save_for_later, how = 'left' , on = 'customer_id')
X_test = X_test.merge(train_pivot, how = 'left' , on = 'customer_id')
X_test = X_test.drop(['customer_id'],axis=1)

model =  xgb.XGBRegressor(learning_rate = 0.1)
model.fit(X_train, Y_train)

val['RF_preds'] = model.predict(X_test)


In [34]:
songs_df=pd.read_csv('songs.csv')
test = val
df_submission = pd.read_csv('dummy_submission.csv')

user_ids = train.customer_id.unique()
total_users = len(user_ids)


### TRUNCSVD ###

#Removing duplicates
song_matrix = pd.concat([train,test]).drop_duplicates(subset = ['customer_id','song_id'],keep = 'first')
#Creates a song matrix of #numofusers vs #noofsongs
song_matrix = song_matrix.pivot('customer_id','song_id','score')

song_means = song_matrix.mean()
user_means = song_matrix.mean(axis=1)
#Mean shifting
song_shifted_temp = song_matrix-song_means
song_shifted = song_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -song_shifted_temp.isnull()

def repeated_matrix_reconstruction(num_pcs,num_iterations):
    global song_shifted
    for i in range(num_iterations):
        SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
        SVD.fit(song_shifted)
        #For the ease of applying masks we work with pandas
        song_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(song_shifted)),columns=song_shifted.columns,index=song_shifted.index)
        loss = mean_squared_error(song_represented[mask].fillna(0),song_shifted_temp[mask].fillna(0))
        print('Iteration: {} , Loss: {} '.format(i,loss))
        #To just update the non-zero values of song_reprented values to the true ratings
        
        if i < (num_iterations - 1):
            song_represented[mask] = song_shifted_temp[mask]
        
        song_shifted = song_represented
            
    #Mean shifting it back
    song_mat = song_shifted + song_means
    song_mat = song_mat.clip(lower=1,upper=5)
    return song_mat
print("Starting truncated svd with number of components as 20")
representative_matrix_20 = repeated_matrix_reconstruction(50,30)
print("Done")
print("Starting truncated svd with number of components as 15")
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
print("Done")
#bagging
rating_matrix = (representative_matrix_15+representative_matrix_20)/2


trunc_prediction = np.zeros(len(test))
for i in range(len(test)):
    userid =  test.iloc[i,0]
    songid = test.iloc[i,1]
    trunc_prediction[i] = rating_matrix[rating_matrix.index==userid][songid].values[0]

val['RM_preds'] = trunc_prediction

Starting truncated svd with number of components as 20
Iteration: 0 , Loss: 0.0037743578078364955 
Iteration: 1 , Loss: 0.003540102287425589 
Iteration: 2 , Loss: 0.0033429451313313125 
Iteration: 3 , Loss: 0.003175441917840218 
Iteration: 4 , Loss: 0.0030311241165732307 
Iteration: 5 , Loss: 0.002904912611749046 
Iteration: 6 , Loss: 0.0027930506148835742 
Iteration: 7 , Loss: 0.002692747972473885 
Iteration: 8 , Loss: 0.0026019073667800895 
Iteration: 9 , Loss: 0.0025189324108215072 
Iteration: 10 , Loss: 0.002442591286633025 
Iteration: 11 , Loss: 0.0023719194087260414 
Iteration: 12 , Loss: 0.0023061500050893203 
Iteration: 13 , Loss: 0.0022446645331124296 
Iteration: 14 , Loss: 0.0021869570599033347 
Iteration: 15 , Loss: 0.0021326084380799246 
Iteration: 16 , Loss: 0.002081267366909296 
Iteration: 17 , Loss: 0.0020326363233682085 
Iteration: 18 , Loss: 0.0019864609683032386 
Iteration: 19 , Loss: 0.0019425220586673359 
Iteration: 20 , Loss: 0.0019006291882907587 
Iteration: 21 , 

In [75]:
X_test = val[['RM_preds','RF_preds']]

X_test = np.clip(X_test, a_min= 1, a_max = 5)
X_test.loc[np.isfinite(X_test['RM_preds']) == False, 'RM_preds'] = X_test.loc[np.isfinite(X_test['RM_preds']) == False, 'RF_preds']
X_test.loc[np.isnan(X_test['RM_preds']), 'RM_preds'] = X_test.loc[np.isnan(X_test['RM_preds']), 'RF_preds']

preds = final_model.predict(X_test)
PRED = np.around(preds,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)

val['preds'] = PRED

In [76]:
test = pd.read_csv('test.csv')

test = test.merge(val[['customer_id','song_id','preds']], how = 'left', on = ['customer_id', 'song_id'])
test.head()

,customer_id,song_id,preds
0,J24644,5304,3.2
1,G29701,123,3.7
2,M29977,11,4.2
3,H50752,1445,3.7
4,J42394,271,4.5


In [77]:
PRED = test.preds
df_submission = pd.read_csv('dummy_submission.csv')
df_submission.score = PRED
df_submission.to_csv('./RM_RF_ensemble_v5.csv',index=False)

In [80]:
val.head()

,customer_id,song_id,First_Customer_id,platform_id,released_year,language,number_of_comments,First_song_id,song_avg,cust_avg,cust_count,cust_var,cust_std,song_count,song_var,song_std,RF_preds,RM_preds,preds
0,J24644,5304.0,J,U64955,1989.0,0,416.0,U,3.468085,3.566667,60,0.859887,0.927301,47.0,0.689177,0.830167,3.229712,3.391751,3.2
1,J24644,1423.0,J,Y10619,1995.0,eng,1714.0,Y,3.428571,3.566667,60,0.859887,0.927301,126.0,1.014857,1.007401,3.119885,2.898422,3.0
2,J24644,670.0,J,S11573,1979.0,en-US,2294.0,S,3.843137,3.566667,60,0.859887,0.927301,255.0,0.676085,0.822244,3.493561,3.214446,3.4
3,J24644,4465.0,J,V653396,1987.0,en-US,672.0,V,3.692308,3.566667,60,0.859887,0.927301,39.0,1.218623,1.103913,3.343798,3.646602,3.3
4,J24644,5.0,J,Q4671,1925.0,eng,51992.0,Q,3.846195,3.566667,60,0.859887,0.927301,1879.0,1.049281,1.024344,3.486274,4.154678,3.6
